In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
import nltk
import nltk.corpus
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder
#from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

[nltk_data] Downloading package punkt to /home/x-akolli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/x-akolli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/x-akolli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [116]:
dataset = pd.read_csv("data.csv")

In [72]:
# def data_clean(entry):
#   # Lowercase the texts
#   entry = entry.lower()

#   # Cleaning punctuations in the text
#   punc = str.maketrans(string.punctuation, ' '*len(string.punctuation))
#   entry = entry.translate(punc)

#   # Removing numbers in the text
#   entry = re.sub(r'\d+', '', entry)

#   # Remove possible links
#   entry = re.sub('https?://\S+|www\.\S+', '', entry)

#   # Deleting newlines
#   entry = re.sub('\n', '', entry)

#   return entry

In [73]:
Stopwords = set(nltk.corpus.stopwords.words("english")) - set(["not"])

In [74]:
# def data_process(entry):
#   Processed_Text = list()
#   Lemmatizer = WordNetLemmatizer()

#   # Tokens of Words
#   Tokens = nltk.word_tokenize(entry)

#   # Removing Stopwords and Lemmatizing Words
#   # To reduce noises in our dataset, also to keep it simple and still 
#   # powerful, we will only omit the word `not` from the list of stopwords

#   for word in Tokens:
#     if word not in Stopwords:
#       Processed_Text.append(Lemmatizer.lemmatize(word))

#   return(" ".join(Processed_Text))

In [75]:
# dataset["Answer"] = dataset["Answer"].apply(lambda Text: data_clean(Text))
# dataset["Answer"] = dataset["Answer"].apply(lambda Text: data_process(Text))

In [117]:
dataset.iloc[:, 1:] = dataset.iloc[:, 1:].astype(int)
df = dataset

In [118]:
nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()
import string

list_pos = []
list_neg = []
list_neu = []
list_compound = []

for row in range (0, df.shape[0]):
    
    example_sent = df.iloc[row]['Answer']
    example_sent = example_sent.replace(r'.', '')
    example_sent = example_sent.replace(r',', '')
    example_sent = example_sent.replace(r'!', '')
    example_sent = example_sent.replace(r';', '')
    example_sent = example_sent.replace(r'?', '')
    example_sent = example_sent.replace(r'(', '')
    example_sent = example_sent.replace(r')', '')
    example_sent = example_sent.replace(r"'", '')
    
    #example_sent = example_sent[0].replace(r'[^\w\s]+,', '')
    stop_words = set(stopwords.words('english'))
    list_contractions_common = {'arent', 'cant', 'couldnt', 'didnt', 'doesnt', 'dont', 'hadnt', 'havent', 'shouldnt', 'wouldnt', 'youve','youre','wont','werent', 'weve','wed', 'theyre', 'Im'}
    stop_words.update(list_contractions_common)

    word_tokens = word_tokenize(example_sent)
    # converts the words in word_tokens to lower case and then checks whether 
    #they are present in stop_words or not
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    #with no lower case conversion
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
            
    filtered_sentence = ' '.join(filtered_sentence)
       
            
    df.at[row,'Answer'] = filtered_sentence
    
    list_pos.append((sia.polarity_scores(filtered_sentence))['pos'])
    list_neg.append((sia.polarity_scores(filtered_sentence))['neg'])
    list_neu.append((sia.polarity_scores(filtered_sentence))['neu'])
    list_compound.append((sia.polarity_scores(filtered_sentence))['compound'])
    
df['positive_score'] = list_pos
df['negative_score'] = list_neg
df['neutral_score'] = list_neu
df['compound_score'] = list_compound

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/x-akolli/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [119]:
df.head()

,Answer,Answer.f1.afraid.raw,Answer.f1.angry.raw,Answer.f1.anxious.raw,Answer.f1.ashamed.raw,Answer.f1.awkward.raw,Answer.f1.bored.raw,Answer.f1.calm.raw,Answer.f1.confused.raw,Answer.f1.disgusted.raw,...,Answer.t1.health.raw,Answer.t1.love.raw,Answer.t1.recreation.raw,Answer.t1.school.raw,Answer.t1.sleep.raw,Answer.t1.work.raw,positive_score,negative_score,neutral_score,compound_score
0,My family salient part day since days care 2 c...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.362,0.038,0.600,0.9274
1,Yoga keeps focused I able take time breath wor...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.253,0.000,0.747,0.5267
2,Yesterday family I played bunch board games My...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.457,0.061,0.481,0.9300
3,Yesterday I visited parents dinner I seen week...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.291,0.000,0.709,0.5719
4,Yesterday I really felt importance health I we...,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0.296,0.188,0.516,0.4998


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1473 entries, 0 to 1472
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Answer                    1473 non-null   object 
 1   Answer.f1.afraid.raw      1473 non-null   int64  
 2   Answer.f1.angry.raw       1473 non-null   int64  
 3   Answer.f1.anxious.raw     1473 non-null   int64  
 4   Answer.f1.ashamed.raw     1473 non-null   int64  
 5   Answer.f1.awkward.raw     1473 non-null   int64  
 6   Answer.f1.bored.raw       1473 non-null   int64  
 7   Answer.f1.calm.raw        1473 non-null   int64  
 8   Answer.f1.confused.raw    1473 non-null   int64  
 9   Answer.f1.disgusted.raw   1473 non-null   int64  
 10  Answer.f1.excited.raw     1473 non-null   int64  
 11  Answer.f1.frustrated.raw  1473 non-null   int64  
 12  Answer.f1.happy.raw       1473 non-null   int64  
 13  Answer.f1.jealous.raw     1473 non-null   int64  
 14  Answer.f

In [121]:
df.rename(columns={"Answer.f1.afraid.raw": "afraid", 
                   "Answer.f1.angry.raw": "angry", 
                   "Answer.f1.anxious.raw": "anxious", 
                   "Answer.f1.ashamed.raw": "ashamed", 
                   "Answer.f1.awkward.raw": "awkward", 
                   "Answer.f1.bored.raw": "bored", 
                   "Answer.f1.calm.raw": "calm", 
                   "Answer.f1.confused.raw": "confused", 
                   "Answer.f1.disgusted.raw": "disgusted", 
                   "Answer.f1.excited.raw": "excited", 
                   "Answer.f1.frustrated.raw": "frustrated", 
                   "Answer.f1.happy.raw": "happy", 
                   "Answer.f1.jealous.raw": "jealous", 
                   "Answer.f1.nostalgic.raw": "nostalgic", 
                   "Answer.f1.proud.raw": "proud", 
                   "Answer.f1.sad.raw": "sad", 
                   "Answer.f1.satisfied.raw": "satisfied", 
                   "Answer.f1.surprised.raw": "surprised", 
                   "Answer.t1.exercise.raw": "exercise", 
                   "Answer.t1.family.raw": "family", 
                   "Answer.t1.food.raw": "food", 
                   "Answer.t1.friends.raw": "friends", 
                   "Answer.t1.god.raw": "god", 
                   "Answer.t1.health.raw": "health", 
                   "Answer.t1.love.raw": "love", 
                   "Answer.t1.recreation.raw": "recreation", 
                   "Answer.t1.school.raw": "school", 
                   "Answer.t1.sleep.raw": "sleep", 
                   "Answer.t1.work.raw": "work",                    
                  }, inplace=True)

In [122]:
# add emotions into one list
final_emotion_list = []
for row in range (0, len(df)):
    emotion_list = []
    for col in range (1, 19):
        if (df.iloc[row][col] == 1):
            emotion_list.append(df.columns[col])
    final_emotion_list.append(emotion_list)

/tmp/ipykernel_4101818/3494996672.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (df.iloc[row][col] == 1):


In [123]:
print(len(final_emotion_list))

1473


In [124]:
print((final_emotion_list))

[['anxious', 'happy', 'proud'], ['calm'], ['calm', 'happy', 'satisfied'], ['calm', 'happy'], ['happy'], ['angry', 'anxious', 'bored', 'frustrated', 'sad'], ['anxious', 'proud', 'satisfied'], ['proud', 'satisfied'], ['happy', 'proud'], ['happy'], ['happy'], ['happy'], ['happy', 'nostalgic', 'proud'], ['excited', 'happy', 'satisfied'], ['excited', 'happy', 'proud'], ['proud'], ['calm', 'happy', 'satisfied'], ['angry', 'frustrated', 'sad'], ['anxious', 'satisfied'], ['happy'], ['excited', 'happy', 'proud'], ['happy'], ['frustrated'], ['bored', 'frustrated', 'satisfied'], ['confused', 'frustrated', 'surprised'], ['happy', 'satisfied'], ['happy', 'satisfied'], ['excited', 'happy', 'satisfied'], ['frustrated'], ['disgusted'], ['excited', 'happy', 'satisfied'], ['confused', 'sad'], ['happy'], ['excited', 'happy'], ['happy', 'proud', 'satisfied'], ['anxious'], ['excited', 'happy', 'proud'], ['satisfied'], ['happy', 'proud', 'satisfied'], ['angry', 'anxious', 'frustrated'], ['proud'], ['calm', 

In [125]:
df['Emotions Felt'] = final_emotion_list

In [126]:
df.head()

,Answer,afraid,angry,anxious,ashamed,awkward,bored,calm,confused,disgusted,...,love,recreation,school,sleep,work,positive_score,negative_score,neutral_score,compound_score,Emotions Felt
0,My family salient part day since days care 2 c...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0.362,0.038,0.600,0.9274,"[anxious, happy, proud]"
1,Yoga keeps focused I able take time breath wor...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0.253,0.000,0.747,0.5267,[calm]
2,Yesterday family I played bunch board games My...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0.457,0.061,0.481,0.9300,"[calm, happy, satisfied]"
3,Yesterday I visited parents dinner I seen week...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0.291,0.000,0.709,0.5719,"[calm, happy]"
4,Yesterday I really felt importance health I we...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.296,0.188,0.516,0.4998,[happy]


In [127]:
# topic sorting into list
final_topic_list = []
for row in range (0, len(df)):
    #topic_list = []
    for col in range (19, 30):
        if (df.iloc[row][col] == 1):
            final_topic_list.append(df.columns[col])

/tmp/ipykernel_4101818/617195242.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (df.iloc[row][col] == 1):


In [128]:
print(len(final_topic_list))
print(final_topic_list)

1471
['family', 'exercise', 'family', 'family', 'health', 'work', 'work', 'school', 'work', 'god', 'love', 'family', 'family', 'exercise', 'work', 'family', 'work', 'health', 'work', 'love', 'family', 'family', 'exercise', 'work', 'recreation', 'health', 'exercise', 'recreation', 'exercise', 'health', 'family', 'health', 'food', 'work', 'family', 'health', 'work', 'work', 'exercise', 'work', 'work', 'work', 'work', 'work', 'work', 'work', 'work', 'work', 'work', 'god', 'family', 'health', 'family', 'love', 'friends', 'work', 'love', 'food', 'family', 'family', 'family', 'exercise', 'food', 'family', 'work', 'work', 'work', 'god', 'family', 'family', 'health', 'work', 'work', 'work', 'work', 'work', 'family', 'recreation', 'friends', 'family', 'work', 'family', 'sleep', 'family', 'love', 'recreation', 'family', 'recreation', 'love', 'god', 'work', 'family', 'food', 'food', 'friends', 'god', 'family', 'family', 'love', 'family', 'family', 'family', 'family', 'family', 'sleep', 'family', 

In [140]:
for row in range (0, len(df)):
    #topic_list = []
    count = 0
    for col in range (19, 30):
        if (df.iloc[row][col] == 1):
            count = count + 1
    if (count == 0):
        print(row)

/tmp/ipykernel_4101818/3941235496.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (df.iloc[row][col] == 1):


In [141]:
print(df.iloc[961][0])
# seems to be because of work
print(df.iloc[961][34])
print(df.iloc[961][33])

Having faith keeps feeling steady good mood
['calm', 'happy', 'proud']
0.7351


/tmp/ipykernel_4101818/3952047956.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[961][0])
/tmp/ipykernel_4101818/3952047956.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[961][34])
/tmp/ipykernel_4101818/3952047956.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[961][33])


In [142]:
print(df.iloc[980][0])
#seems to be because of god
print(df.iloc[980][34])
print(df.iloc[980][33])

Yesterday I went walk 2 kids 2 dogs My 2-year old great usually difficult time kept held hand almost whole time The 7-month old fell asleep Both dogs well-behaved
['happy', 'satisfied']
0.7003


/tmp/ipykernel_4101818/1085653319.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[980][0])
/tmp/ipykernel_4101818/1085653319.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[980][34])
/tmp/ipykernel_4101818/1085653319.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[980][33])


In [143]:
df.iat[961,29] = 1
print(df.iloc[961][29])

1


/tmp/ipykernel_4101818/1845222933.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[961][29])


In [144]:
#now let's add the labels into the dataframe
#second one is for god which is col 23
print(df.iloc[980][23])
df.iat[980,23] = 1
print(df.iloc[980][23])

0
1


/tmp/ipykernel_4101818/2655515971.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[980][23])
/tmp/ipykernel_4101818/2655515971.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[980][23])


In [145]:
#Now, let's rerun the code to get all the topics and the length should be correct
#11 topics
#columns 18-29 (inclusive of both)
#no topic lists greater than one for one person/entry, so we can just append it to one list 
final_topic_list = []
for row in range (0, len(df)):
    #topic_list = []
    for col in range (19, 30):
        if (df.iloc[row][col] == 1):
            final_topic_list.append(df.columns[col])
#no topic lists greater than one, so 

/tmp/ipykernel_4101818/558654078.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (df.iloc[row][col] == 1):


1471